# Battery RUL Estimation using AI Methods
This notebook follows the course workflow and uses the provided dataset for RUL prediction with a Transformer-based model.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Load dataset
url = 'https://raw.githubusercontent.com/Aftermarked/battery-rul-prognostics/main/Battery_RUL.csv'
df = pd.read_csv(url)
df.head()


In [ ]:

# Data filtering (based on instructions)
df = df[(df['F1'] > 500) & (df['F1'] < 2500)]
df = df[(df['F5'] > 3.7) & (df['F5'] < 4.1)]
df = df[df['F6'] < 4.05]
df = df[df['F3'] < 7000]

# Interpolation of missing values
df.interpolate(method='linear', inplace=True)
df.dropna(inplace=True)

# Min-max normalization
min_ = df.min()
max_ = df.max()
df_norm = (df - min_) / (max_ - min_)

# Features and target
X = df_norm.drop(columns=['Cycle Index', 'RUL']).values
y = df_norm['RUL'].values


In [ ]:

# Reshape for sequence input: [samples, timesteps, features]
# Assuming window of 20 steps
sequence_length = 20
X_seq, y_seq = [], []

for i in range(len(X) - sequence_length):
    X_seq.append(X[i:i+sequence_length])
    y_seq.append(y[i+sequence_length])

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

# Train/test split
split = int(len(X_seq) * 0.8)
x_train, x_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]


In [ ]:

input_layer = tf.keras.layers.Input(shape=x_train.shape[1:])
mha = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=100)(input_layer, input_layer)
conv1d = tf.keras.layers.Conv1D(filters=100, kernel_size=3, padding='same', activation='relu')(mha)
dropout = tf.keras.layers.Dropout(rate=0.2)(conv1d)
flatten = tf.keras.layers.Flatten()(dropout)
output_layer = tf.keras.layers.Dense(units=1, activation='linear')(flatten)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5, decay=1e-5), loss='mse', metrics=['mae'])


In [ ]:

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
history = model.fit(x_train, y_train, epochs=20, batch_size=32, callbacks=[es], validation_split=0.2)


In [ ]:

yp_test = model.predict(x_test)
yp_test = yp_test * (max_['RUL'] - min_['RUL']) + min_['RUL']
y_test = y_test * (max_['RUL'] - min_['RUL']) + min_['RUL']

plt.plot(yp_test, label='Pred')
plt.plot(y_test, label='Meas')
plt.xlim(0, 820)
plt.ylim(70, 92)
plt.xlabel('Cycle Index')
plt.ylabel('SOH (%)')
plt.legend()
plt.grid(True)
plt.title("Prediction vs Measurement")
plt.show()


In [ ]:

epoch = history.epoch
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(epoch, train_loss)
plt.title("Training Loss")

plt.subplot(1, 2, 2)
plt.plot(epoch, val_loss)
plt.title("Validation Loss")

plt.tight_layout()
plt.show()
